In [1]:
import numpy as np
from datasets import load_dataset

dataset = load_dataset("brainer/pill_identification_data")
dataset

DatasetDict({
    train: Dataset({
        features: ['품목일련번호', '품목명', '업소일련번호', '업소명', '성상', '큰제품이미지', '표시앞', '표시뒤', '의약품제형', '색상앞', '색상뒤', '분할선앞', '분할선뒤', '크기장축', '크기단축', '크기두께', '이미지생성일자(약학정보원)', '분류번호', '분류명', '전문일반구분', '품목허가일자', '제형코드명', '표기내용앞', '표기내용뒤', '표기이미지앞', '표기이미지뒤', '표기코드앞', '표기코드뒤', '변경일자', '사업자번호', 'Unnamed: 30'],
        num_rows: 25330
    })
})

In [2]:
dataset = dataset["train"]
dataset = dataset.select_columns(['품목일련번호', '크기장축', '크기단축', '크기두께'])


# Drop 품목명 == '보넥실캅셀<소브레롤>'
def drop_not_float(example):
    """
    크기장축, 크기단축, 크기두께가 float으로 변환 가능한지 확인
    :param example: 
    :return: 
    """
    try:
        float(example["크기장축"])
        float(example["크기단축"])
        float(example["크기두께"])
        return True
    except ValueError:
        return False


dataset = dataset.filter(drop_not_float)
dataset

Dataset({
    features: ['품목일련번호', '크기장축', '크기단축', '크기두께'],
    num_rows: 25221
})

In [3]:
# dtype str to int
dataset = dataset.map(
    lambda example: {
        "크기장축": np.asarray(example["크기장축"]).astype(np.float64),
        "크기단축": np.asarray(example["크기단축"]).astype(np.float64),
        "크기두께": np.asarray(example["크기두께"]).astype(np.float64),
    },
    num_proc=7,
    batched=True,
    batch_size=1000,
    keep_in_memory=True,
)

Map (num_proc=7):   0%|          | 0/25221 [00:00<?, ? examples/s]

In [4]:
# Node: 품목일련번호: 식품의약품안전처 약품식별정보및 상세정보 보기 원료약품 및 분량, 효능효과, 사용상의주의사항,재심사, RMP, 보험, 기타정보, 생산실적
# Edge: 크기장축, 크기단축, 크기두께를 고려한 상대적 부피

# 부피
dataset = dataset.add_column(name="크기", column=list(np.asarray(dataset["크기장축"]) * np.asarray(dataset["크기단축"])))

In [5]:
dataset.column_names

['품목일련번호', '크기장축', '크기단축', '크기두께', '크기']

In [6]:
from tqdm.notebook import tqdm
from pyvis.network import Network

# dataset = dataset.shard(num_shards=100, index=0).to_pandas()

# '크기' 열의 값들을 리스트로 추출
size_list = dataset['크기']

# 가정: '품목일련번호' 열의 값들이 리스트로 주어짐
item_numbers = dataset['품목일련번호']

# pyvis의 Network 객체 생성
net = Network(notebook=True)

# 노드 추가
for item in tqdm(item_numbers):
    net.add_node(item)

edge_count = {item: 0 for item in item_numbers}

# 엣지 추가
for i, item_i in tqdm(enumerate(item_numbers), total=len(item_numbers)):
    for j, item_j in enumerate(item_numbers):
        if item_i != item_j and edge_count[item_i] < 3:
            # 여기서 size_list를 사용하여 가중치를 계산합니다.
            # size_list가 0인 경우를 처리하기 위한 조건문을 추가할 수 있습니다.
            weight = size_list[i] / size_list[j] if size_list[j] != 0 else 0
            net.add_edge(item_i, item_j, value=weight)
            edge_count[item_i] += 1  # 에지 수 업데이트

  0%|          | 0/25221 [00:00<?, ?it/s]

  0%|          | 0/25221 [00:00<?, ?it/s]

In [7]:
net.write_html("size-graph.html")
# net.show("size-graph.html")

In [21]:
from datasets import Dataset
import pandas as pd

# 네트워크 데이터를 리스트 형식으로 변환
nodes_data = [{'node': node, 'label': str(node)} for node in net.nodes]
edges_data = [{'value': edge['value'], 'from': edge['from'], 'to': edge['to']} for edge in net.edges]

# pandas DataFrame을 Hugging Face datasets.Dataset으로 변환
nodes_dataset = Dataset.from_pandas(pd.DataFrame(nodes_data))
edges_dataset = Dataset.from_pandas(pd.DataFrame(edges_data))
nodes_dataset, edges_dataset

(Dataset({
     features: ['node', 'label'],
     num_rows: 25205
 }),
 Dataset({
     features: ['value', 'from', 'to'],
     num_rows: 75609
 }))

In [20]:
nodes_dataset.push_to_hub("brainer/pill_identification_graph", config_name="size-graph-nodes")
edges_dataset.push_to_hub("brainer/pill_identification_graph", config_name="size-graph-edges")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/76 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/502 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/brainer/pill_identification_graph/commit/7d966055cc18c19b97bbf2f597de2491d5e37159', commit_message='Upload dataset', commit_description='', oid='7d966055cc18c19b97bbf2f597de2491d5e37159', pr_url=None, pr_revision=None, pr_num=None)